In [1]:
import gym
from gym import spaces
import numpy as np
from pprint import pprint
import random
from simplified_connectx_ppo import *


In [2]:
def p_print(board):
    temp = board
    temp = np.array(temp)
    temp = temp.reshape(6,7)
    pprint(temp)

#### When swapping the board, also swap the top and current player, done, info . They contain the state of the game.

In [3]:
env = SimplifiedConnectXPPO()

In [5]:
def play_random_game(env):
    env.reset()
    while not env.done:
        possible_actions = env.get_possible_actions()
        
        action = random.choice(possible_actions)
        obs,rew,d,info = env.step(action)
#         p_print(obs)
#         print(rew)
#         print()
        

In [6]:
import time
start = time.time()
# 1000 games / sec for random
for i in range(1000):
    play_random_game(env)
    
end = time.time() - start
print("total time", end)

total time 0.9862208366394043


In [7]:
env.top

[5, 3, 1, 1, 1, 0, 4]

In [8]:
arr = [0]*42

arr[7*6 - 2] = 1
arr[7*5 - 2] = 1
arr[7*4 - 2] = 1
arr[7*3 - 2] = 1

arr[7*6 - 1] = 2
arr[7*5 - 1] = 2
arr[7*4 - 1] = 2
arr[7*3 - 1] = 1

arr[(6 - 1)*(7-1) + 0] = 1

In [9]:
p_print(arr)

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 1, 2],
       [0, 0, 1, 0, 0, 1, 2],
       [0, 0, 0, 0, 0, 1, 2]])


In [10]:
env.is_win(7,6,4,arr,6,2)

False

In [11]:
arr = [0] *42
top = [0]*7
for i in range(6):
    for j in range(7):
        arr[(42) - top[j] * 7 - (7 - j)] = (i)*7 + (j+1)
        top[j] += 1


In [12]:
p_print(arr)

array([[36, 37, 38, 39, 40, 41, 42],
       [29, 30, 31, 32, 33, 34, 35],
       [22, 23, 24, 25, 26, 27, 28],
       [15, 16, 17, 18, 19, 20, 21],
       [ 8,  9, 10, 11, 12, 13, 14],
       [ 1,  2,  3,  4,  5,  6,  7]])


In [13]:
top

[6, 6, 6, 6, 6, 6, 6]

In [14]:
arr[29]

9

In [113]:
arr[42 - 7]

1

In [8]:
arr = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2,
0, 0, 0, 1, 2, 2, 1, 1, 0, 0, 1, 2]
arr2 = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 2,
0, 0, 0, 1, 2, 2, 1, 1, 0, 0, 1, 2]

In [9]:
p_print(arr)
p_print(arr2)

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [2, 2, 0, 0, 0, 1, 2],
       [2, 1, 1, 0, 0, 1, 2]])
array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0],
       [2, 2, 0, 0, 0, 1, 2],
       [2, 1, 1, 0, 0, 1, 2]])


### Lets make a few puzzles to test how well the NN and the algorithm is doing

##### Win in one puzzle

In [15]:
import logging
import os
import sys
from collections import deque
from pickle import Pickler, Unpickler
from random import shuffle

import numpy as np
from tqdm import tqdm

from Arena import Arena
from MCTS import MCTS
from Game import Game
from NeuralNet import *
from Coach import *
from pprint import pprint
from utils import *

In [24]:
class PureNN():
    '''
    This is the final agent in test mode
    
    '''
    def __init__(self,load_folder_file):
#       The location of where the model is stored
        self.load_folder_file = load_folder_file
        self.nn_args = {
        'lr': 0.001,
        'dropout': 0.3,
        'epochs': 100,
        'batch_size': 64,
        'cuda': True,
        'num_channels': 128,
        'num_residual_layers': 20
        }
        self.game = Game()
        self.nn = NeuralNet(self.game)
        self.load_model()
        
        
    def load_model(self):
        self.nn.nnet.model.load_weights(self.load_folder_file)
            
    def get_best_action(self,board):
        
        self.game.env.board = board
        cannonical = self.game.getCanonicalForm()
        action = self.nn.predict(cannonical)
        
        return action

In [39]:
weights_location = r"C:\Users\TheDarkAce\Jupyter scripts\COMP 552 RL\Project\Simplified connectX\temp2\temp.h5"
pure = PureNN(weights_location)

In [40]:
arr = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 1, 1]
p_print(arr)

array([[0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [2, 2, 2, 0, 0, 1, 1]])


In [41]:
pure.get_best_action(arr)

(array([0.10954547, 0.07764035, 0.02893143, 0.1603642 , 0.33768496,
        0.21115717, 0.0746764 ], dtype=float32),
 array([-0.2861795], dtype=float32))